# 1. Importing modules

In [8]:
import cv2 # used to perform computer vision tasks
import numpy as np # used to work with arrays
import os # used to navigate through directories
from facenet_pytorch.models.mtcnn import MTCNN #face-detection model
import torch # helpful in building, training and using ML and DL models

In [ ]:
pip uninstall facenet-pytorch

Found existing installation: facenet-pytorch 2.6.0
Uninstalling facenet-pytorch-2.6.0:
  Would remove:
    /home/mukesh/.local/lib/python3.10/site-packages/facenet_pytorch-2.6.0.dist-info/*
    /home/mukesh/.local/lib/python3.10/site-packages/facenet_pytorch/*
Proceed (Y/n)? 

# 2. Setting up face-detection model

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# sets 'GPU' or 'CPU' based on the hardware availability
mtcnn = MTCNN(keep_all = True, device = device)
# Multi-task Cascaded Convolution Neural Networks.
# keep_all parameter retains all detected faces and facial landmarks in the output

In [10]:
def mtcnn_detect(img: np.ndarray) -> np.ndarray:
    boxes, probs = mtcnn.detect(img)
#     boxes = bounding boxes around an image, probs = the confidence score of it being a face
    img = []
    
    if boxes is not None:
        for box in boxes :
            x_left = int(min(box[0], box[2]))
            x_right = int(max(box[0], box[2]))
            y_left = int(min(box[1], box[3]))
            y_right = int(max(box[1], box[3]))
            img.append([x_left, y_left, x_right, y_right])
            
    # returning the collection of detected faces along with their confidence scores
    return (img, probs)

# 3. Creating new directory for new dataset

In [11]:
# Creating a new directory
folderName = input("Enter the person name : ")
path = os.path.join('Datasets', folderName)

# check whether directory already exists
if not os.path.exists(path):
  os.mkdir(path)
  print(f"Folder {folderName} created!")
else:
  print(f"Folder {folderName} already exists!")

Enter the person name : lucky
Folder lucky created!


# 4. Collecting faces through webcam/video

In [12]:
cap = cv2.VideoCapture(0) # 0=camera | videofile also accepted(just give the name)
cnt = 0

while True:
    ret, frame = cap.read()
    
    if(ret == False):
        print("Not able to read image...Exiting...")
        exit()
    else:
        facesBoxes, confidenceScores = mtcnn_detect(frame)
        
        text = str(cnt)
        textColor = (0, 255, 0)
        if(len(facesBoxes) == 0):
            text = "No faces found !"
            textColor = (0, 0, 255)
        elif(len(facesBoxes) > 1):
            text = "Only one person at a time !"
            textColor = (0, 0, 255)
        else:
            for faceBox, score in zip(facesBoxes, confidenceScores):
                if(score >= 0.95):
                    cnt += 1
                    
                    x_left, y_left, x_right, y_right = faceBox
                    
                    croppedImg = frame[y_left:y_right, x_left:x_right]
                    croppedImg = cv2.resize(croppedImg, (224, 224))
                    imgPath = os.path.join(path, (str(cnt)+".jpg"))
                    cv2.imwrite(imgPath, croppedImg)
                    
                    cv2.rectangle(frame, (x_left, y_left), (x_right, y_right), (0, 255, 0), 2)
    
        cv2.putText(frame, text, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, textColor, 2)
        cv2.imshow("Video", frame)
        
    if(cv2.waitKey(1) == ord('q') or cnt >= 100):
        break
        
cap.release()
cv2.destroyAllWindows()

In [13]:
# In case the video window is not closed run this cell
cap.release()
cv2.destroyAllWindows()